In [1]:
!pip install transformers==4.38.2
!pip install numpy==1.26.4
!pip install datasets==2.15
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.0
    Uninstalling fsspec-2024.3.0:
      Successfully uninstalled fsspec-2024.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Found e

In [2]:
import pandas as pd
import torch
 
from transformers import T5Tokenizer,T5ForConditionalGeneration,TrainingArguments, Trainer

from datasets import Dataset
from sklearn.model_selection import train_test_split

2024-03-21 08:31:57.452985: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 08:31:57.453084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 08:31:57.647631: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
MODEL = 't5-small'
BATCH_SIZE = 32
NUM_PROCS = 4
EPOCHS = 3
OUT_DIR = '/kaggle/tmp/'
MAX_LENGTH = 256

In [4]:
df = pd.read_csv("/kaggle/input/smart-swiping-test/test.csv")
df = df[["simulated","text"]]
df

,simulated,text
0,iuytgfgtyuioiuytfdedfcfghyuiuygfdertyuioiuytre...,If you decide to eat
1,hgtrerertyuhjuytresdertygbvgfre,here just be
2,ghuioijnbhgy,going
3,iuyt,it
4,wasdfghnjhyt,want
...,...,...
99995,tyghytrewertyuioiuytrtyuikoiuyt,the workout
99996,rewasdfghjkloiuyuijkmkiuytfdsdfgtyuijnjhgtyhyt...,really missing the spiritual
99997,oiuytftghgftre,of the
99998,dfghjuioiuytrewertghn,down


In [5]:
# df['word_count'] = df['text'].str.split().apply(len)
# df['word_count']

In [6]:
# df[df["word_count"] == 1]

In [7]:
# import matplotlib.pyplot as plt

# plt.hist(df['word_count'])
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')
# plt.show()

In [8]:
train_df, val_df = train_test_split(df,train_size = 0.90)
train_df

,simulated,text
9973,loiuytrerewasdfghjklkjiuyujuytrfdsert,LITERALLY just
99974,rtyuiuytrewesdfghjuiuytredsertyuiklkjhgf,fish itself
88458,wasdrtyuiuytyuijnjhg,waiting
22432,tyghgfrertyuiopoiuygfdszsawertyuiokjhgfgtyuioi...,The pizza took forrrevvverrr to
56799,ewasdcfghujnjiuytrewqwertyuytrerfghbhgfrertygh...,each unique beer have
...,...,...
73058,fdsasdfvbnmkiolkjiuytfrtyuiuytrertghbnbvgfdfgh...,Family friendly
96900,oiuytrdered,ordered
58736,oiuygfcvghjkoiuytresewertyuioiuioklkjhgfdfghjh...,of course wouldn't be
75486,tyuytrertytrfdsertyuioiuytrdsdfghjklkiuyuhjnjh...,They seriously need to have


In [9]:
val_df

,simulated,text
780,oiuytrertyujklkjhgfdsertyuioiuytrewertyuioklkj...,or else you will wait
61624,sertyuiopoiuytred,stopped
35385,a,a
29538,iuyghjuioiuytrewas,I got was
66189,tyghgytrerfgvbnjiolokjhytre,Then I looked
...,...,...
38761,frtyuiuytrerfghjnjhgfrtyuioklkjiuy,friendly
46141,okjnjkloiuy,Only
50068,ghuioiuytfdsertfgvghjuiuygfcfde,Good service
93775,oiuytyhgtredfcdsasdxcvbnbhgfdrtytrerfghjiopokj...,out the candy shop but


In [10]:
train_df = Dataset.from_pandas(train_df)
val_df = Dataset.from_pandas(val_df)
print(train_df)
print(val_df)

Dataset({
    features: ['simulated', 'text', '__index_level_0__'],
    num_rows: 90000
})
Dataset({
    features: ['simulated', 'text', '__index_level_0__'],
    num_rows: 10000
})


In [11]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)
tokenizer

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5Tokenizer(name_or_path='t5-small', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_4

In [12]:
def preprocess_function(batch):
    model_inputs = tokenizer( batch["simulated"] ,max_length = MAX_LENGTH, truncation=True, padding='max_length' )
    labels = tokenizer( batch["text"], max_length=MAX_LENGTH, truncation=True, padding='max_length'  )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:

train_data = train_df.map(
    preprocess_function, 
    batched=True, 
    batch_size = BATCH_SIZE, 
    num_proc=NUM_PROCS,
    remove_columns = ["simulated", "text"],

)
train_data

Map (num_proc=4):   0%|          | 0/90000 [00:00<?, ? examples/s]

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 90000
})

In [14]:

val_data = val_df.map(
    preprocess_function, 
    batched=True, 
    batch_size = BATCH_SIZE, 
    num_proc=NUM_PROCS,
    remove_columns = ["simulated", "text"],

)
val_data

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [15]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [16]:

total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(  p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

60,506,624 total parameters.
60,506,624 training parameters.


In [17]:
train_df[0]

{'simulated': 'loiuytrerewasdfghjklkjiuyujuytrfdsert',
 'text': 'LITERALLY just',
 '__index_level_0__': 9973}

In [18]:
90000 / BATCH_SIZE

2812.5

In [19]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    evaluation_strategy='steps',
    save_steps=500,
    eval_steps=500,
    load_best_model_at_end=True,
    save_total_limit=5,
    report_to='tensorboard',
    learning_rate=0.0001,
    fp16=True,
    dataloader_num_workers=4
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset = val_data,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [20]:
history = trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
500,0.066500,0.062778
1000,0.058700,0.052569
1500,0.054700,0.047575
2000,0.049800,0.044523
2500,0.047600,0.042522
3000,0.045600,0.041127
3500,0.046900,0.040328
4000,0.046500,0.039872


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [21]:
history

TrainOutput(global_step=4221, training_loss=0.2754999751026979, metrics={'train_runtime': 6218.5313, 'train_samples_per_second': 43.419, 'train_steps_per_second': 0.679, 'total_flos': 1.827114319872e+16, 'train_loss': 0.2754999751026979, 'epoch': 3.0})

In [23]:
model.save_pretrained("/kaggle/working/model")

In [24]:
tokenizer.save_pretrained("/kaggle/working/tokenizer")

('/kaggle/working/tokenizer/tokenizer_config.json',
 '/kaggle/working/tokenizer/special_tokens_map.json',
 '/kaggle/working/tokenizer/spiece.model',
 '/kaggle/working/tokenizer/added_tokens.json')

In [25]:
model_path = '/kaggle/working/model'
trained_model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained('/kaggle/working/tokenizer')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
def infer(text, model, tokenizer):
    inputs = tokenizer.encode(
        text,
        return_tensors='pt',
        max_length=256,
        padding='max_length',
        truncation=True
    )
 
    corrected_ids = model.generate(
        inputs,
        max_length=256,
        num_beams=5,
        early_stopping=True
    )
 
    corrected_sentence = tokenizer.decode(
        corrected_ids[0],
        skip_special_tokens=True
    )
    return corrected_sentence

In [27]:
infer("hjioiuytrewaswerertyuioiu", trained_model, tokenizer)

'he was you'

In [28]:
infer("hgferghjklo", trained_model, tokenizer)

'hello'